# Module 1 - Langchain Core
## 1.1 Setup and LLMChain


> Goals
  1.   Install Langchain
  2.   Use LCEL
  3.   Test LCEL

To install Langchain, use  ```pip install langchain```
We will use llama with Groq.



In [ ]:
pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Setting up environment
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("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")

# imorting dependencies
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

# setting up llm,prompt template and chain
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Translate this to French: {text}")
chain = prompt | model

This is a chain to translate into french. Prompt template takes the promps and pass it through llm to get output.

In [ ]:
chain.invoke({"text": "Hello, how are you?"})

AIMessage(content='Bonjour, comment vas-tu?\n\n(Note: "vas-tu" is the informal way of saying "how are you" in French, used with friends or people you\'re familiar with. If you want to use the formal way, you can say "comment allez-vous?")', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 21, 'total_tokens': 79, 'completion_time': 0.128706761, 'prompt_time': 0.006431708, 'queue_time': 0.006583062999999999, 'total_time': 0.135138469}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_8b7c3a83f7', 'finish_reason': 'stop', 'logprobs': None}, id='run--8b31263d-9b32-4484-8fbd-b8631d243f35-0', usage_metadata={'input_tokens': 21, 'output_tokens': 58, 'total_tokens': 79})

Let us build LCEL chain that takes a product name and returns a company slogan.

In [ ]:
# imorting dependencies
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

# setting up llm,prompt template and chain
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Write a company slogan for this {product}")
chain = prompt | model

In [ ]:
response = chain.invoke({"product": "eco water bottle"})
print(response)

content='Here are a few company slogan ideas for the eco water bottle:\n\n1. **"Hydrate sustainably, every sip"** - emphasizing the eco-friendly aspect of the product.\n2. **"Drink up, waste less"** - highlighting the reduction of single-use plastic waste.\n3. **"Pure water, pure conscience"** - appealing to customers who care about the environment.\n4. **"Sip, save, repeat"** - a playful way to encourage customers to keep using the eco-friendly bottle.\n5. **"Quench your thirst, not the planet\'s"** - a clever phrase that resonates with environmentally conscious consumers.\n6. **"Bottle up the good stuff, not the waste"** - emphasizing the benefits of using a reusable, eco-friendly water bottle.\n7. **"Elevate your hydration, elevate the earth"** - a slogan that promotes a sense of responsibility and eco-friendliness.\n8. **"The sip that saves the planet, one bottle at a time"** - a bold statement that encourages customers to make a positive impact.\n\nChoose the one that resonates wi

### Using Output parser
Output parser is a technique to get a structured output. The basic syntax is ```StrOutputParser ``` .
 There are several more options  ```JsonOutputParser``` , ```PydanticOutputParser ```.

In [ ]:
# imorting dependencies
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# setting up llm,prompt template,parser and chain
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Write exactly one short company slogan for this {product}")
parser = StrOutputParser()
chain = prompt | model |parser

response = chain.invoke({"product": "eco water bottle"})
print(response)

"Sip Smart, Earth Happy"


### Preprocessing Input
Sometimes we have to process our input too. Users give inputs in a hapazhard way. On these cases we also have to modify inputs too.

For these cases, we have to build function and implement it with ``` runnable ```.

Now let us make a preprocessor which is also set our tone as catchy with product name. So we have to add anaother element tone with value catchy in our dictionary.

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# defining function
def add_tone(input : dict) -> dict:
  input["tone"]= {"funny"}
  return input

# building preprocessor with runnable
preprocess = RunnableLambda(add_tone)

# building propmpt and model
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Write exactly one short company slogan for this {product} with a {tone} tone.")
parser = StrOutputParser()

# building chain
chain = preprocess | prompt  |model | parser

In [ ]:
# Run
result = chain.invoke({"product": "blood"})
print(result)

"Blood you can bank on... and also use to make a killer Bloody Mary"


### Adding tools with LCEL
To process different tasks we have to corporate more tools in our chain. Let us build a tool such that it will take weights in kg and return it in lbs. And then we will implement it with our chain.

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# defining weight tool
def weight(input : dict) -> dict:
  weight = input["weight"]
  weight_lbs = round(weight * 2.20462, 2)
  return {
      "product": input["product"],
      "weight_in_lbs": weight_lbs
  }

# building preprocessor with runnable
weight_lbs = RunnableLambda(weight)

# building propmpt and model
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Write exactly one short company slogan for this {product} with a {weight_in_lbs} weight.")
parser = StrOutputParser()

# building chain
chain = weight_lbs | prompt  |model | parser

In [ ]:
# Run
result = chain.invoke({"product": "blood","weight" : 0})
print(result)

"Sip, don't sin: Zero-Calorie Blood for a Clear Conscience"


### Building a Modular Multi-Step Chain
So now we will differrent tools in one chain to do differrent work so that it we can logic in different chain in a multi-chain system.

So we will build a small AI Village Assistant System.
We will not use any agent. We will use LCEL only.

So our chain will divide our query into three categories - symptom advice,nutritional tip and rest suggestions. We will need 4 chain - 1 classification chain and 3 chains for 3 types of outputs.


In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# classification chain
model = init_chat_model("llama3-8b-8192", model_provider="groq")
prompt = ChatPromptTemplate.from_template("Classify input : {input} into exactly any one of these category- ['Symptom','Diet','Rest'].If you cannot classify input  return 'other'.Just return the category only.")
parser = StrOutputParser()
classifier_chain = prompt | model | parser

# Symptom tool
sym_prompt = ChatPromptTemplate.from_template("You are one of the best medical consultant. Study the symptoms : {input} and give safe and hepful short advice. Do not include anything more.")
sym_chain = sym_prompt | model | parser

# Diet tool
diet_prompt = ChatPromptTemplate.from_template("You are one of the best dietician. Study the query:{input} and give compact short helpful nutritious diet advice only.")
diet_chain = diet_prompt | model | parser

# Rest tool
rest_prompt = ChatPromptTemplate.from_template("Based on users query:{input} give some helpful advices to rest.")
rest_chain = rest_prompt | model | parser

# Fallback chain
default_chain = RunnableLambda(lambda x: "Sorry, I don't know how to respond to that.")

# branch chain
branch_chain = RunnableBranch(
    (lambda x:"Symptom" in x["type"],sym_chain),
    (lambda x:"Diet" in x["type"],diet_chain),
    (lambda x:"Rest" in x["type"],rest_chain),
    default_chain
    )

# combining all stages
def classify_and_route(input:dict)->dict:
  classification = classifier_chain.invoke({"input":input})

  return branch_chain.invoke({
      "input":input,
      "type":classification.strip()
  })

In [ ]:
result_1 = classify_and_route({"input":"I feel very cold and have headache"})
print(result_1)

Based on the symptoms, it's possible that you may be experiencing a common cold or flu. Here's a short advice:

* Drink plenty of warm liquids such as tea, coffee, or broth to help raise your body temperature and ease the cold sensation.
* Take an over-the-counter pain reliever such as acetaminophen or ibuprofen to help alleviate your headache.
* Rest and avoid strenuous activities to help your body recover.

Please consult a healthcare professional if your symptoms worsen or persist.


In [ ]:
result_2 = classify_and_route({"input":"What food can I eat during weakness?"})
print(result_2)

Thank you for reaching out! When you're feeling weak, it's essential to fuel up with nutrient-rich foods that can help boost your energy levels. Here are some compact and helpful advice:

**Eat these foods:**

1. **Bananas**: Rich in potassium, bananas help regulate fluid balance and electrolytes, which can help alleviate fatigue.
2. **Leafy Greens**: Spinach, kale, and collard greens are packed with iron, which is essential for healthy red blood cells and energy production.
3. **Nuts and Seeds**: Almonds, cashews, pumpkin seeds, and sunflower seeds are rich in healthy fats, protein, and fiber, which can help stabilize blood sugar levels and provide a energy boost.
4. **Fatty Fish**: Fatty fish like salmon, tuna, and mackerel are rich in omega-3 fatty acids, which can help reduce inflammation and support energy production.
5. **Sweet Potatoes**: Rich in complex carbohydrates, sweet potatoes provide sustained energy and are easy to digest.
6. **Avocados**: Avocados are a rich source of 

In [ ]:
result_3 = classify_and_route({"input":"My sleep is disturbed due to stress"})
print(result_3)

Based on the symptom "My sleep is disturbed due to stress", I would advise:

"Take 10-15 minutes each day to practice relaxation techniques such as deep breathing, progressive muscle relaxation, or guided imagery to help calm your mind and body. Establish a consistent sleep schedule and create a bedtime routine to signal your body that it's time to sleep. Avoid screens and stimulating activities at least an hour before bedtime. Consider keeping a stress journal to identify and manage stressors in your life."


In [ ]:
result_4 = classify_and_route({"input":"djbnfjdjsbjjaduhufjdkjjzm md mz mmmmmmmmmmsmmmmm"})
print(result_4)

Sorry, I don't know how to respond to that.
